# Non Linear Programing (NLP)

Same structure as Linear Programing (LP) but with the difference that:

$ minf(x)$

subject to

$ h(x) = 0 $

$ g(x) <= 0 $

- **The main difference is that f,h or g are non linear**
- x is all continuous variable

## Example 1

consider the nonlinear regression problem using the data x and y, where *y* is the dependent variable and *x* is the independent variable

- 6 observations/data points are avilable fot this regression model.

- The required model structure is $y= b_1 + b_2e^{b_3x}$

|y          |x         |
|-----------|----------|
|127        |-5        |
|151        |-3        |
|379        |-1        |
|421        |5         |
|460        |3         |
|426        |1         |


**OBJ: Find the optimal values of regression coefficients $b_1,b_2, b_3$**